In [1]:
import os, sys, glob

print(os.getcwd())
os.chdir('..')
print(f'Changed dir to: {os.getcwd()}')

/Users/shayanriyaz/Documents/Projects/heart_rhythm_analysis/notebooks
Changed dir to: /Users/shayanriyaz/Documents/Projects/heart_rhythm_analysis


In [2]:
import os
import numpy as np
import sys
import h5py
import pandas as pd
from IPython.display import display
from random import choice

project_root = os.path.abspath(os.path.join(os.getcwd()))

src_dir = os.path.join(project_root, "src")
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

from src.lib.create_training_pipelines.capnobase_etl import CapnoBaseETL
from src.lib.create_training_pipelines.mimic_etl import MimicETL
from src.lib.utils.timeseries_viewer import make_window_figure, create_time_series_viewer
from notebooks.load_as_df import load_as_df

bSetUpDB = True
WINDOW_LEN = 8

import dash
app = dash.Dash(__name__)

# MIMIC Exploration

In [3]:
def main():
    num_subjects = 10
    out_path = os.path.join(f'data/development_dataset/length_full/')
    out_filename = os.path.join(f'train_dataset')
    df = load_as_df(out_path,out_filename,num_subjects=10)
    return df


In [4]:
specs = [
        {"key":"raw_ppg",  "fs_key":"raw_ppg_fs",  "subplot":1, "legend":"Raw PPG"},
        {"key":"proc_ppg", "fs_key":"ppg_fs_out",  "subplot":1, "legend":"Proc PPG"},
        {"key":"raw_ekg",  "fs_key":"ekg_fs_out",  "subplot":2,               },
        {"key":"raw_abp",  "fs_key":"abp_fs_out",  "subplot":3,               },
    ]

if __name__ == "__main__": 
    df_mimic = main()
#     # first = df_mimic3_gen.iloc[np.random.randint(0, len(df_mimic3_gen))] 
#     chosen_subj = df_mimic['subject'].unique()[0]

#     surr_subject_df = df_mimic[df_mimic['subject'] == chosen_subj]

#     display(surr_subject_df.head())


#     fig_fn = lambda window: make_window_figure(window, specs,surr_subject_df)
    
#     app = create_time_series_viewer(
#     surr_subject_df,
#     fig_fn,
#     specs,
#     index_label_fn=lambda idx, row: f"{row.subject}-{row.window_count}"
# )
#     print(f'Total # of Subjects: {len(df_mimic['subject'].unique())}')
#     print(f'Total # of Windows: {df_mimic.shape[0]}')
#     print("Launching Dash at http://127.0.0.1:8050 …")
#     app.run(debug=True,port = 8080)

Loading: data/development_dataset/length_full//train_dataset.h5


In [5]:
subject_agg = {
    # sampling & metadata: just take the first (they're constant per subject)
    "raw_ppg_fs": "first",
    "ppg_fs_out": "first",
    "ekg_fs_out": "first",
    "abp_fs_out": "first",
    # lengths & durations: sum across windows
    "raw_len":       "sum",
    "proc_len":      "sum",
    "duration_raw_s":  "sum",
    "duration_proc_s": "sum",
    # signals: concatenate all windows end-to-end
    "raw_ppg":  lambda s: np.concatenate(s.values),
    "proc_ppg": lambda s: np.concatenate(s.values),
    "raw_ekg":  lambda s: np.concatenate(s.values),
    "raw_abp":  lambda s: np.concatenate(s.values),
    # if you want to keep track of window_ids or rec_ids:
    "window_id": lambda s: list(s.values),
    "rec_id":    lambda s: list(s.values),
    "label":     lambda s: list(s.values),
}

df_by_subject = (
    df_mimic
    .groupby("subject", as_index=False)
    .agg(subject_agg)
)

print(df_by_subject.shape)
print(df_by_subject.raw_ppg.iloc[0].shape)  

fig_fn = lambda row: make_window_figure(row, specs)
app = create_time_series_viewer(
    df_by_subject,
    fig_fn,
    specs,
    index_label_fn=lambda idx, row: str(row.subject)
)
print("Launching Dash at http://127.0.0.1:8080 …")
app.run(debug=True,port=8080)

(10, 16)
(153000,)
Launching Dash at http://127.0.0.1:8080 …


In [ ]:
#     specs = [
#         {"key":"raw_ppg",  "fs_key":"raw_ppg_fs",  "subplot":1, "legend":"Raw PPG"},
#         {"key":"proc_ppg", "fs_key":"ppg_fs_out",  "subplot":1, "legend":"Proc PPG"},
#         {"key":"raw_ekg",  "fs_key":"ekg_fs_out",  "subplot":2,               },
#         {"key":"raw_abp",  "fs_key":"abp_fs_out",  "subplot":3,               },
#     ]

#     fig_fn = lambda window: make_window_figure(window, specs,surr_subject_df)
    
#     app = create_time_series_viewer(
#     df_mimic,
#     fig_fn,
#     specs,
#     index_label_fn=lambda idx, row: f"{row.subject}-{row.window_count}"
# )
#     print(f'Total # of Subjects: {len(df_mimic['subject'].unique())}')
#     print(f'Total # of Windows: {df_mimic.shape[0]}')
#     print("Launching Dash at http://127.0.0.1:8050 …")
#     app.run(debug=True,port = 8080)

In [ ]:
# subject_agg = {
#     # sampling & metadata: just take the first (they're constant per subject)
#     "raw_ppg_fs": "first",
#     "ppg_fs_out": "first",
#     "ekg_fs_out": "first",
#     "abp_fs_out": "first",
#     # lengths & durations: sum across windows
#     "raw_len":       "sum",
#     "proc_len":      "sum",
#     "duration_raw_s":  "sum",
#     "duration_proc_s": "sum",
#     # signals: concatenate all windows end-to-end
#     "raw_ppg":  lambda s: np.concatenate(s.values),
#     "proc_ppg": lambda s: np.concatenate(s.values),
#     "raw_ekg":  lambda s: np.concatenate(s.values),
#     "raw_abp":  lambda s: np.concatenate(s.values),
#     # if you want to keep track of window_ids or rec_ids:
#     "window_id": lambda s: list(s.values),
#     "window_count": lambda s: list(s.values),
#     "rec_id":    lambda s: list(s.values),
#     "label":     lambda s: list(s.values),
#     "notes": lambda s: list(s.values),
# }

# # ── 2) group by subject ─
# df_by_subject = (
#     df_mimic
#     .groupby("subject", as_index=False)
#     .agg(subject_agg)
# )

# # ── 3) now df_by_subject has one row per subject, with each signal a long array
# print(df_by_subject.shape)
# print(df_by_subject.raw_ppg.iloc[0].shape)  # e.g. (sum of all its windows,) 

# # bind specs & figure fn exactly as before
# fig_fn = lambda row: make_window_figure(row, specs)

# # subject‐level dashboard (labels are just the subject IDs)
# app = create_time_series_viewer(
#     df_by_subject,
#     fig_fn,
#     specs,
#     index_label_fn=lambda idx, row: str(row.subject)
# )
# # print(df_by_subject['notes'])
# print("Launching Dash at http://127.0.0.1:8080 …")
# app.run(debug=True,port=8020)